## Run this cell to set up paths, imports, and W&B logging

In [1]:
%cd ..

import os
from PIL import Image

import wandb
import numpy as np
import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import src.model
from src.training import train, validate

/home/mytkom/Documents/iml


## 1. Initialize W&B project

In [2]:
wandb.init(project="iml", config={
    "learning_rate": 0.001,
    "epochs": 80,
    "batch_size": 32,
    "image_size": (32, 32) # TODO: choose best image_size
})
config = wandb.config

# Configure directories
TRAIN_DIR = "./datasets/train/"
VAL_DIR = "./datasets/val/"
TEST_DIR = "./datasets/test/"

# Speakers of such labels should be admitted to enter
VALID_ACCESS_LABELS = {
    "f1", "f7", "f8", "m3", "m6", "m8"
}

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mytkom (mytkom-warsaw-university-of-technology). Use `wandb login --relogin` to force relogin


## 2. Define a custom dataset class to load spectrograms from files

In [3]:
class SpectrogramDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = self.files[idx]
        speaker_id = img_path.split('/')[-1].split('_')[0]
        label = int(speaker_id in VALID_ACCESS_LABELS)
        image = Image.open(img_path).convert("L")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


## 3. Set up data transformations and data loaders

In [4]:
transform = transforms.Compose([
    transforms.Resize(config.image_size),
    transforms.ToTensor(),
    # TODO: normalization
])

train_dataset = SpectrogramDataset(TRAIN_DIR, transform=transform)
val_dataset = SpectrogramDataset(VAL_DIR, transform=transform)
test_dataset = SpectrogramDataset(TEST_DIR, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)


## 4. Initialize model, optimizer, and set device

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = src.model.SimpleCNN().to(device)
model = src.model.TutorialCNN() # - for 32x32 images
model.device = device
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)


## 5. Training and validation loop

In [6]:

for epoch in range(config.epochs):
    print(f"Epoch {epoch+1}/{config.epochs}")
    
    train(model, train_loader, criterion, optimizer, epoch, wandb.log, len(train_loader) // 5 - 1)
    recall, precision, val_f1 = validate(model, val_loader)
    wandb.log({"validation/recall": recall, "validation/precision": precision, "validation/f1": val_f1, "epoch": epoch+1})
    print("Recall: ", recall, ", Precision: ", precision, ", Validation F1: ", val_f1)


Epoch 1/80


/home/mytkom/Documents/iml/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall:  0.0 , Precision:  0.0 , Validation F1:  0.38267148014440433
Epoch 2/80


/home/mytkom/Documents/iml/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall:  0.0 , Precision:  0.0 , Validation F1:  0.38267148014440433
Epoch 3/80


/home/mytkom/Documents/iml/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall:  0.0 , Precision:  0.0 , Validation F1:  0.38267148014440433
Epoch 4/80


/home/mytkom/Documents/iml/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall:  0.0 , Precision:  0.0 , Validation F1:  0.38267148014440433
Epoch 5/80
Recall:  0.012307692307692308 , Precision:  0.4 , Validation F1:  0.3930312075983718
Epoch 6/80
Recall:  0.0 , Precision:  0.0 , Validation F1:  0.38222543352601157
Epoch 7/80
Recall:  0.018461538461538463 , Precision:  0.42857142857142855 , Validation F1:  0.3984430975387774
Epoch 8/80
Recall:  0.04923076923076923 , Precision:  0.5925925925925926 , Validation F1:  0.42763422144865443
Epoch 9/80
Recall:  0.2523076923076923 , Precision:  0.6666666666666666 , Validation F1:  0.5705159044600407
Epoch 10/80
Recall:  0.16 , Precision:  0.8 , Validation F1:  0.525
Epoch 11/80
Recall:  0.3076923076923077 , Precision:  0.7194244604316546 , Validation F1:  0.6095782365639011
Epoch 12/80
Recall:  0.4461538461538462 , Precision:  0.7837837837837838 , Validation F1:  0.6926470588235294
Epoch 13/80
Recall:  0.4646153846153846 , Precision:  0.8531073446327684 , Validation F1:  0.7180153557953617
Epoch 14/80
Recall:  0.28

## 6. Save the model

In [ ]:

model_path = "./models/simple_cnn.pth"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
torch.save(model.state_dict(), model_path)
wandb.save(model_path)
wandb.finish()

print("Training complete and model saved!")
